# Problema  4

Calcular las coordenadas horizontales de la estrella Altair (&alpha; Aql) con coordenadas:

RA 19h50m47  DEC +08° 52′ 06″

en Madrid, con **latitud 40° 25′N  y longitud 3° 41′ 31″ W** 

cuando el tiempo sidéreo local es **TSL = 22h12m18s**

## Una nota respecto al tiempo

Para resolver el problema completo, necesitamos calcular a qué hora en tiempo civil o universal corresponde nuestro TSL. Veremos cómo hacerlo en el tema dedicado al cálculo del tiempo. De momento asumiremos el TSL como dado.



## Planteamiento

Los problemas de cambio de coordenadas exigen pasar, normalmente, a través de varios sistemas.

En este caso tenemos **ecuatorial -> horario -> horizontal**.

Siempre es conveniente tener cierta idea de la configuración de la esfera celeste para nuestro ejemplo. En este caso nos encontramos en el hemisferio norte, con lo que los objetos que culminen lo harán en la parte sur del meridiano del observador. Además, la estrella tiene declinación positiva, así que se encontrará entre el **ecuador celeste y polo norte**.

![sphere1](../_static/p11/spherecardinalpoints.png)

### Ecuatorial a horario

El cambio de coordenas ecuatoriales a horarias es simplemente:

`TSL = RA + HA`

y la coordenada de declinación es la misma.

El único paso es convertir todas las coordenadas a una medida angular homogénea y dar el resultado entre 0-24h


In [ ]:
from astropy.coordinates import Angle
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ra = Angle("19h50m47") 
dec = Angle("+08d52m06s")
lat = Angle("40d25m")
lon = Angle("-3d41m31s")
# 2019-09-15 22:48:38.200 UTC
TSL = Angle("22h12m18s")

In [ ]:
ha = TSL - ra
print(ha)

Las coordenadas horarias serán entonces:
 
 * **DEC**: +08d52m06s
 * **H**: 2h21m31s
 
Dado que en coordenadas horarias el sur es 0h y el oeste 6h, la estrella se encuentra en algún lugar entre esos dos puntos

## Coordenadas horarias a horizontales

El cambio sistema horario a horizontal requiere una rotación de ejes. De manera alternativa se puede utilizar trigonometría esférica para resolver el triángulo común formado por objeto, el polo norte y el cenit. Hay que tener cuidado, además, con la convención para el origen del ángulo de acimut. 

Las ecuaciones son (para acimut desde el Sur):

$$
\begin{eqnarray}
\sin a & =&  \sin \phi \sin \delta  + \cos \phi \cos \delta \cos H  \\
\cos a \sin A & = & \cos \delta \sin H \\
\cos a \cos A & = & -\cos \phi \sin \delta + \sin \phi \cos \delta \cos H
\end{eqnarray}
$$

Dado que `A` va de 0 a 360, necesitamos el seno y el coseno para determinar el cuadrante correcto del ángulo.

In [ ]:
sin_a = np.sin(lat) * np.sin(dec) + np.cos(lat) * np.cos(dec) * np.cos(ha)

In [ ]:
a = Angle(np.arcsin(sin_a))

In [ ]:
cos_a = np.cos(a)
print(cos_a)

In [ ]:
sin_A = (np.cos(dec) * np.sin(ha)) / cos_a
cos_A = (-np.cos(lat) * np.sin(dec) + np.sin(lat) * np.cos(dec) * np.cos(ha)) / cos_a

In [ ]:
print(sin_A, cos_A)

Dado que los dos valores son positivos, el resultado es directamente:

In [ ]:
A = Angle(np.arctan(sin_A / cos_A))


In [ ]:
print("a=", a.to_string(unit=u.degree), "A=",A.to_string(unit=u.degree))

Esta medida de acimut tiene como referencia SWNE. Si el convenio de acimut fuera NESW, habría que sumar 180 grados (y restar 360 si fuera necesario)

In [ ]:
A_N = A + Angle("180d")
print(A_N.to_string(unit=u.degree))

## Usando rotaciones de ejes

Otra manera de resolver el problema es utilizando rotaciones de ejes coordenados. Esta manera es más fácil para una computadora, que no tiene más que realizar productos matriciales.

Para ello partimos de las coordenadas horarias y las convertimos en cartesianas.

$$
\begin{eqnarray}
x &=& \cos \delta \cos H \\
y &=& \cos \delta \sin H\\
z &=& \sin \delta
\end{eqnarray}
$$

Nótese que este sistema de coordenadas está orientado según la regla de la mano izquierda, con el eje `x` hacia el sur , el `y` hacia el oeste y el `z` hacia el cenit. El ángulo `H` se incrementa en el sentido de las agujas del reloj.

![sphereaxis](../_static/p11/sphereaxis.png)


In [ ]:
cos_dec = np.cos(dec)
sin_dec = np.sin(dec)
cos_H = np.cos(ha)
sin_H = np.sin(ha)

In [ ]:
r = (cos_dec * cos_H, cos_dec * sin_H, sin_dec)

El sistema horizontal será *diestro* o *zurdo* dependiendo del convenio de acimut. Si el ángulo va del sur al oeste, seguirá el mismo convenio que las coordenadas horarias, luego estará **orientado a izquierdas**.

$$
\begin{eqnarray}
x' &=& \cos a \cos A \\
y' &=& \cos a \sin A\\
z' &=& \sin a
\end{eqnarray}
$$

Ambos sistemas tienen el eje `Y` común. Para pasar del sistema `XYZ` al `X'Y'Z'` hay que realizar una rotación de $\psi = (90 - \phi)$ en sentido horario entorno al eje `Y`.

El cambio de origen del acimut se puede incluir también como una rotación adicional en el eje `Z`, de 180 grados si el origen es el norte en lugar del sur.

Para ello basta con crear al matriz de rotación y realizar un producto matricial:

$ r' = R_z(180) *  R_y(\psi) r $ 


In [ ]:
# Una función de python, para crear diferentes matices de rotación en x

def rot_x(ang):
    return [[1,0,0], 
            [0, np.cos(ang), np.sin(ang)], 
            [0, -np.sin(ang), np.cos(ang)]
           ]

def rot_y(ang):
    return [ 
            [np.cos(ang), 0, -np.sin(ang)],
            [0, 1, 0],
            [np.sin(ang), 0, np.cos(ang)]
           ]

def rot_z(ang):
    return [[np.cos(ang), np.sin(ang), 0], 
            [-np.sin(ang), np.cos(ang), 0],
            [0, 0, 1]
           ]

In [ ]:
psi = (Angle(90, unit=u.deg) - lat)
ry = rot_y(psi)
rz = rot_z(Angle(180, unit=u.deg))

In [ ]:
# los términos con Quantity corresponden a que hemos
# pasado un objeto Angle en lugar de un ángulo en radianes
print(ry)

In [ ]:
# dot es la función de numpy para realizar productos de matrices
rp = np.dot(rz, np.dot(ry, r))

In [ ]:
# vector en el espacio primado
rp

In [ ]:
# Las tres componentes
xp, yp, zp = rp

In [ ]:
a_2 = Angle(np.arcsin(zp), unit=u.rad)

In [ ]:
cos_a_2 = np.cos(a_2)

In [ ]:
sin_A_2 = yp / cos_a_2
cos_A_2 = xp / cos_a_2
print(sin_A, cos_A)
print(sin_A_2, cos_A_2)

El cambio de signo se debe al cambio de origen en azimut:

$$
\begin{eqnarray}
\sin(A + 180) &=& -\sin A \\
\cos(A + 180) &=& -\cos A
\end{eqnarray}
$$


El resultado es el mismo que se calcula con las ecuaciones de triángulos esféricos.

In [ ]:
# Aquí utilizamos la función arctan2(x, y)
A_2 = Angle(np.arctan2(sin_A_2, cos_A_2))

In [ ]:
print("a=", a.to_string(unit=u.degree), "A=",A.to_string(unit=u.degree))

In [ ]:
print("a2=", a_2.to_string(unit=u.degree), "A2=",(A_2+ Angle(360, unit=u.deg)).to_string(unit=u.degree))